In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///Resources\hawaii.sqlite")


In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)
#print(session)


# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results - OK

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [24]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

# Use Inspector to print the column names and types of measurement
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

print("-"*12)
print(engine.execute('SELECT * FROM measurement LIMIT 2').fetchall())

print("-"*12)
# Use Inspector to print the column names and types of station
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

print("-"*12)
engine.execute('SELECT * FROM station LIMIT 2').fetchall()

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT
------------
[(1, 'USC00519397', '2010-01-01', 0.08, 65.0), (2, 'USC00519397', '2010-01-02', 0.0, 63.0)]
------------
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT
------------


[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6)]

In [11]:
#PREVIEW: # Display the row's columns and data in dictionary format
Mrow = session.query(Measurement).first()  # bring the 1st row of the class MEASUREMENT
print(Mrow.__dict__)  
Srow = session.query(Station).first()  # bring the 1st row of the class STATIOM
print(Srow.__dict__)


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001E186A32908>, 'date': '2010-01-01', 'tobs': 65.0, 'id': 1, 'station': 'USC00519397', 'prcp': 0.08}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000001E186A170C8>, 'station': 'USC00519397', 'longitude': -157.8168, 'latitude': 21.2716, 'name': 'WAIKIKI 717.2, HI US', 'id': 1, 'elevation': 3.0}


In [42]:
#LIST ALL EXISTING DATES
#datetype =session.query(Measurement)
#for i in datetype:
#   print(i.date)


# Use the session to query Measurement table and display the first 5 dates
#for row in session.query(Measurement, Measurement.date).limit(5).all():
#    print(row)

#Cuantas FECHA unicas hay
#datetype = session.query(Measurement).group_by(Measurement.date).count()
#print(datetype)

In [53]:
#PREV - change date from text to datetime
#import datetime as dt

#dt.datetime.strptime(date,format)

In [10]:
# Total dates
session.query(func.count(Measurement.date)).all()

[(19550)]

In [11]:
#Last 12 dates register
session.query(Measurement.date).order_by(Measurement.date.desc()).limit(12).all()


[('2017-08-23'),
 ('2017-08-23'),
 ('2017-08-23'),
 ('2017-08-23'),
 ('2017-08-22'),
 ('2017-08-22'),
 ('2017-08-22'),
 ('2017-08-21'),
 ('2017-08-21'),
 ('2017-08-21'),
 ('2017-08-21'),
 ('2017-08-20')]

In [12]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

#session.query(Measurement.date).filter_by(date = '2015%').all()  # NULL result
#session.query(Measurement).filter(Measurement.date == '2015%').all()  # NULL result

#last12months = session.query(Measurement).filter(Measurement.date == '2010-01-01').all() 
#print(last12months)  # --> muestra "objects"  --> Le falta .date en el query

#session.query(Measurement).filter(Measurement.date == '2010-01-01').all() # --> muestra "objects", le falta .date 
#session.query(Measurement.date).filter(Measurement.date == '2010-01-01').all() # --> muestra lista de registros


#session.query(Measurement.date).order_by(Measurement.date).first()

# GET THE LAST DATE REGISTER IN THE DB  
lastdate = session.query(Measurement.date).order_by(Measurement.date.desc()).first().date
lastdate

'2017-08-23'

In [15]:
# DATE IS SAVED AS A STRING. NEED TO BE CHANGED TO DATE FORMAT
lastdateD = dt.datetime.strptime(lastdate,'%Y-%m-%d')
print(lastdateD)
lastdateD

2017-08-23 00:00:00


datetime.datetime(2017, 8, 23, 0, 0)

In [18]:
## Calculate the date 1 year ago from the last data point in the database
# Find date from last 12 months from LastDate
last12monthsS = lastdateD - dt.timedelta(days=365)
print(last12monthsS)
last12monthsS

2016-08-23 00:00:00


datetime.datetime(2016, 8, 23, 0, 0)

In [14]:
#last12months = dt.datetime.strptime(lastdate,'%Y-%m-%d') - dt.timedelta(days=365)
#print(last12months)
#last12months

In [24]:
# Perform a query to retrieve the data and precipitation scores
#Design a query to retrieve the last 12 months of precipitation data 
precipLast12Months = session.query(Measurement.date,Measurement.prcp).all()
session.query(Measurement.date,Measurement.prcp).limit(5).all()

# the results is a list of tuples of (date,precip). Some are none values

[('2010-01-01', 0.08),
 ('2010-01-02', 0.0),
 ('2010-01-03', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-06', None)]

In [33]:
# Save the query results as a Pandas DataFrame and set the index to the date column
# 1. First, extract the tuples values
#datesA = [precipLast12Months[1] for d in precipLast12Months[:10]]
#datesA
precipP = [precipLast12Months[3] for d in precipLast12Months[:10]]
precipP
#precipitation12months = pd.read_sql(precipLast12Months)
#precipitation12months

[('2010-01-04', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-04', 0.0),
 ('2010-01-04', 0.0)]

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
